In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os
from pandas.core.dtypes.common import is_numeric_dtype
from sklearn.preprocessing import MinMaxScaler

In [ ]:
DATA_FOLDER = 'dataset'
DATASET = os.path.join(DATA_FOLDER, 'preprocessed_df.csv')
LABELLED_OUTLIERS = os.path.join(DATA_FOLDER, 'outliers_labelled.csv')
OUTLIERS_REMOVED = os.path.join(DATA_FOLDER, 'outliers_removed.csv')

In [ ]:
df = pd.read_csv(DATASET)

numerical_attr_list = [col for col in df.columns if is_numeric_dtype(df[col])]
categorical_attr_list = [col for col in df.columns if not is_numeric_dtype(df[col])]

## Preliminary

In [ ]:
# Redefine some stuff of used OD classes
# to obtain code continuity
from sklearn.neighbors import LocalOutlierFactor
from pyod.models.abod import ABOD
from sklearn.ensemble import IsolationForest

class MyLOF(LocalOutlierFactor):
    name = 'LOF'

    @property
    def scores(self): return self.negative_outlier_factor_

class MyABOD(ABOD):
    name = 'ABOD'

    @property
    def scores(self): return self.decision_scores_

class MyISFO(IsolationForest):
    name = 'ISFO'

    def fit(self, X, **kwargs):
        self._train_samp = X
        return super().fit(X)
    
    @property
    def scores(self): return self.score_samples(self._train_samp)

## Find outliers

For each method finds the top `p%` outliers

In [ ]:
p = 0.02

methods = [MyLOF, MyABOD, MyISFO]
params = [dict(n_jobs=-1), dict(), dict(n_jobs=-1, n_estimators=100)] # Parameters of the estimators
params

In [ ]:
# The sub-samples in which the dataset is divided
# Currently the OD is performed on each element of (vocal_channel, sex) product category
locality_areas = []
for vc in ['song', 'speech']:
    for s in ['M','F']:
        locality_areas.append(dict(vocal_channel=vc, sex=s))

locality_areas

In [ ]:
# for loop on each combination of outliers classifier and parameters
for meth, pars in zip(methods, params):

    # Initializes each record not to be an outlier
    df[f'is_{meth.name}_outlier'] = False

    for locality in locality_areas:

        # locality_mask: row i == True iff the corresponding row in the dataset matches
        # the dictionary values under consideration (i.e., the values of sex and vc)
        locality_mask = np.array([(df[key] == val) for key, val in locality.items()]).all(axis=0)

        # np.sum(locality_mask) -> number of rows that belong to the same category
        #                          (again, 4 categories grouped by sex and vc)
        #
        # N_p -> number of outliers to be detected for the category (p = 0.02 -> always 2%)
        N_p = int(p * np.sum(locality_mask))
        print(f'{meth.name}: Taking {N_p} outliers for locality {list(locality.keys())} = {list(locality.values())}',
              end='')

        # subsamp -> numeric columns of the dataset objects that belong to the category under consideration
        subsamp = df[numerical_attr_list][locality_mask].values
        subsamp = MinMaxScaler().fit_transform(subsamp)

        # get an instance of the estimator (meth) passing the parameters (pars) as arguments
        outlier_detector = meth(**pars)
        outlier_detector.fit(subsamp)

        # save scores into the dataframe
        df.loc[locality_mask, f'{meth.name}_score'] = outlier_detector.scores

        # takes the indexes of the worsts
        bad_kids = df.loc[locality_mask, f'{meth.name}_score'].nsmallest(N_p).index
        print(f' --> {bad_kids.values}')

        # set them to be outliers
        df.loc[bad_kids, f'is_{meth.name}_outlier'] = True

In [ ]:
# assigns a different color based on whether it is an outlier or not, and the method by which it was detected
df['outlier_color'] = 'k'
code_map = dict(not_outlier='k', ABOD='g', LOF='r', ISFO='b')
for meth in methods:
    df.loc[df[f'is_{meth.name}_outlier'], 'outlier_color'] = code_map[meth.name]

Now check whether the outliers have a predominant category

In [ ]:
def plot_polar_closed(items, r, ax, **kwargs):
    items = list(items)
    asort = np.argsort(items)

    items = np.array(items)[asort]
    r = np.array(list(r))[asort]

    theta = np.linspace(0, 6.28, len(items), endpoint=False)

    theta = np.append(theta, theta[0])
    r = np.append(r, r[0])
    fill_kwargs = kwargs.copy()
    fill_kwargs['alpha'] = 0.6,
    fill_kwargs['label'] = ''

    ax.fill_between(theta,0*r, r, **fill_kwargs)
    ax.plot(theta, r, **kwargs)

    ax.set_xticks(theta[:-1], items)
    ax.set_rticks([])

fig, axes = plt.subplots(1,2, figsize=(10,8), subplot_kw={'projection': 'polar'})
for i, cat in enumerate(['emotion', 'actor']):
    cat_vals, cat_counts = np.unique(df[cat], return_counts=True)
    if cat =='actor':
        cat_vals = [int(act_name.split('_')[1]) for act_name in cat_vals]
    cat_counts = cat_counts/np.sum(cat_counts)

    for meth in methods:
        outlying_items, counts_num = np.unique(df.loc[df[f'is_{ meth.name}_outlier'], cat], return_counts=True)
        counts_num = counts_num/np.sum(counts_num)
        if cat == 'actor':
            outlying_items = [int(act_name.split('_')[1]) for act_name in outlying_items]

        counts = {c:0 for c in np.unique(cat_vals)}
        for outlying_item, out_count, cc in zip(outlying_items, counts_num, cat_counts):
            # Normalize on category count
            # e.g.: More values of fear --> more outliers of fear
            counts[outlying_item] = out_count/cc

        plot_polar_closed(counts.keys(), counts.values(), axes[i], label=meth.name, color=code_map[meth.name])

axes[i].legend()
axes[0].set_title('emotion')
axes[1].set_title('actor');

In [ ]:
binary_categories = ['emotional_intensity',  'statement', 'repetition', 'sex', 'vocal_channel',]
fig, axes = plt.subplots(1, len(binary_categories), figsize=(10, 4), sharey=True)
for cat, ax in zip(binary_categories, axes):
    small_df = pd.DataFrame()
    for meth in methods:
        o_values, o_counts = np.unique(df.loc[df[f'is_{meth.name}_outlier'], cat], return_counts = True)
        a_values, a_counts = np.unique(df[cat], return_counts = True)
        small_df = pd.concat([small_df, pd.DataFrame(dict(  method=[meth.name]*len(a_values), 
                                                            cat=cat,
                                                            rel_counts=o_counts/a_counts,
                                                            attr=a_values))])
        
    obj = sns.barplot(data=small_df,  x='method', y='rel_counts', ax=ax, hue='attr', alpha=0.8)
    ax.legend([], [], frameon=False)
    ax.set_title(cat)
    ax.set_ylabel('')
plt.tight_layout()

## Check into an embedding

In [ ]:
from sklearn.manifold import Isomap
from umap import UMAP
reducer = UMAP(n_components=2, n_neighbors=100)
embedding = reducer.fit_transform(MinMaxScaler().fit_transform(df[numerical_attr_list]))

In [ ]:
plt.scatter(*embedding.T, color="k", s=5)
plt.scatter(*(embedding[df.outlier_color != "k"]).T, c=df.outlier_color[df.outlier_color != "k"], alpha=0.6, s=50)

from matplotlib.patches import Patch
leg_handles, leg_lab = [], []
for k,v in code_map.items():
    leg_handles.append(Patch(color=v))
    leg_lab.append(k)
plt.legend(leg_handles, leg_lab);

## TEST: Outliers on embedding

In [ ]:
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler
n_comp = 2
reduced_columns = [f'fake_attr{i}' for i in range(n_comp)]

reducer = PCA(n_components=n_comp)
X = reducer.fit_transform(MinMaxScaler().fit_transform(df[numerical_attr_list]))

df_reduced = pd.DataFrame(list(X), columns=reduced_columns)
df_reduced = df_reduced.join(df[categorical_attr_list])

In [ ]:
p = 0.01

methods = [MyLOF, MyABOD, MyISFO]
params = [dict(n_jobs=-1),# LOF
          dict(method='fast', n_neighbors=15, contamination=p),# ABOD
          dict(n_jobs=-1, n_estimators=200)# ISFO
          ] # Parameters of the estimators

for meth, pars in zip(methods, params):
    # Initializes each record not to be an outlier
    df_reduced[f'{meth.name}_score'] = 70
    df_reduced[f'is_{meth.name}_outlier'] = False
    for locality in locality_areas:
        locality_mask = np.array([(df_reduced[key] == val).values for key, val in locality.items()]).all(axis=0)
        N_p = int(p*np.sum(locality_mask))
        print(f'{meth.name}: Taking {N_p} outliers for locality {list(locality.keys())} = {list(locality.values())}', end='')

        subsamp = MinMaxScaler().fit_transform( df_reduced.loc[locality_mask][reduced_columns].values )
        outlier_detector = meth(**pars)
        outlier_detector.fit(subsamp)
        df_reduced.loc[locality_mask, f'{meth.name}_score'] = outlier_detector.scores

        # Takes the worsts
        bad_kids = df_reduced.loc[locality_mask, f'{meth.name}_score'].nsmallest(N_p).index
        print(f' --> {bad_kids.values}')
        # Set them to be outliers
        df_reduced.loc[bad_kids,f'is_{meth.name}_outlier'] = True
        
df_reduced['outlier_color'] = 'k'
code_map = dict(not_outlier='k', ABOD='g', LOF='r', ISFO='b')
for meth in methods:
    df_reduced.loc[df_reduced[f'is_{meth.name}_outlier'], 'outlier_color'] = code_map[meth.name]

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(10,8), subplot_kw={'projection': 'polar'})
for i, cat in enumerate(['emotion', 'actor']):
    cat_vals, cat_counts = np.unique(df[cat], return_counts=True)
    if cat =='actor':
        cat_vals = [int(act_name.split('_')[1]) for act_name in cat_vals]
    cat_counts = cat_counts/np.sum(cat_counts)

    for meth in methods:
        outlying_items, counts_num = np.unique(df.loc[df[f'is_{ meth.name}_outlier'], cat], return_counts=True)
        counts_num = counts_num/np.sum(counts_num)
        if cat == 'actor':
            outlying_items = [int(act_name.split('_')[1]) for act_name in outlying_items]

        counts = {c:0 for c in np.unique(cat_vals)}
        for outlying_item, out_count, cc in zip(outlying_items, counts_num, cat_counts):
            # Normalize on category count
            # e.g.: More values of fear --> more outliers of fear
            counts[outlying_item] = out_count/cc

        plot_polar_closed(counts.keys(), counts.values(), axes[i], label=meth.name, color=code_map[meth.name])

axes[i].legend()
axes[0].set_title('emotion');
axes[1].set_title('actor');

In [ ]:
plt.scatter(X[:,0], X[:,1], color='k', s=5)
plt.scatter(X[df_reduced.outlier_color != 'k',0], X[df_reduced.outlier_color != 'k',1], c=df_reduced.outlier_color[df_reduced.outlier_color != 'k'], alpha=0.6, s=50)

from matplotlib.patches import Patch
leg_handles, leg_lab = [], []
for k,v in code_map.items():
    leg_handles.append(Patch(color=v))
    leg_lab.append(k)
plt.legend(leg_handles, leg_lab);

## TEST 2: OD Aggregation

In [ ]:
df_bag = df.copy()
print(len(df_bag))

In [ ]:
# Redefine some stuff of used OD classes
# to obtain code continuity
from sklearn.neighbors import LocalOutlierFactor
from pyod.models.abod import ABOD
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler

class AggregatedOD(object):
    name = 'aggregatedOD'
    def __init__(self, coefficient_vector=None, **pars):
        self.methods = []
        if coefficient_vector is None:
            coefficient_vector = np.ones(3)/3
        else:
            self.coefficient_vector = np.array(coefficient_vector)/np.sum(coefficient_vector)

        for od_meth in [MyISFO, MyABOD, MyLOF]:
            self.methods.append(od_meth(** (pars.get(od_meth.name, dict() ))))

    def fit(self, X):
        for od_meth in self.methods:
            od_meth.fit(X)

    @property
    def scores(self):
        self.single_scores = [MinMaxScaler().fit_transform(od_meth.scores.reshape(-1,1)).reshape(-1) for od_meth in self.methods]
        self.single_scores = np.array(self.single_scores)
        return self.coefficient_vector[None, :].dot(self.single_scores).reshape(-1)

In [ ]:
p = 0.01

params = dict(ISFO=dict(n_jobs=-1, n_estimators=100),
              ABOD=dict(),
              LOF=dict(n_jobs=-1), ) # Parameters of the estimators

meth = AggregatedOD
outlier_detector = meth(coefficient_vector=[0.3, 1,1], 
                        **params)

# Initializes each record not to be an outlier
df_bag[f'is_{meth.name}_outlier'] = False
df_bag[f'{meth.name}_score'] = 70

for locality in locality_areas:
    locality_mask = np.array([(df_bag[key] == val).values for key, val in locality.items()]).all(axis=0)
    N_p = int(p*np.sum(locality_mask))
    print(f'{meth.name}: Taking {N_p} outliers for locality {list(locality.keys())} = {list(locality.values())}', end='')

    subsamp = MinMaxScaler().fit_transform( df_bag.loc[locality_mask][numerical_attr_list].values )
    outlier_detector.fit(subsamp)
    df_bag.loc[locality_mask, f'{meth.name}_score'] = outlier_detector.scores
    for m,s in zip(outlier_detector.methods, outlier_detector.single_scores):
        df_bag.loc[locality_mask, f'{m.name}_score'] = s


    # Takes the worsts
    bad_kids = df_bag.loc[locality_mask, f'{meth.name}_score'].nsmallest(N_p).index
    print(f' --> {bad_kids.values}')
    # Set them to be outliers
    df_bag.loc[bad_kids,f'is_{meth.name}_outlier'] = True


In [ ]:
for col in [u for u in df_bag.columns if u.endswith("score")]:
    plt.hist(df_bag[col], bins=50, alpha=0.5, label=col);
plt.legend()
plt.yscale("log")

In [ ]:
plt.scatter(df_bag["ABOD_score"], df_bag["LOF_score"],label="ABOD vs LOF", s=3, alpha=0.6)
plt.scatter(df_bag["ABOD_score"], df_bag["ISFO_score"],label="ABOD vs ISFO", s=3, alpha=0.6)
plt.scatter(df_bag["LOF_score"], df_bag["ISFO_score"],label="LOF vs ISFO", s=3, alpha=0.6)

plt.plot(np.linspace(0,1), np.linspace(0,1), ls=":", color="k", label="perfect accordance")
plt.legend()

In [ ]:
reducer = Isomap(n_components=2, n_neighbors=15)
X = reducer.fit_transform(df_bag[numerical_attr_list])
plt.scatter(X[:,0], X[:,1], c=df_bag.aggregatedOD_score, cmap="viridis", s=10)
mask = df_bag["is_aggregatedOD_outlier"]
plt.scatter(X[mask, 0], X[mask, 1], color="r", marker="x")

In [ ]:
code_map = dict(not_outlier="k", ABOD="g", LOF="r", ISFO="b", aggregatedOD="k")

fig, axes = plt.subplots(1,2, figsize=(10,8), subplot_kw={'projection': 'polar'})
for i, cat in enumerate(["emotion", "actor"]):
    cat_vals, cat_counts = np.unique(df_bag[cat], return_counts=True)
    if cat =="actor":
        cat_vals = [int(act_name.split("_")[1]) for act_name in cat_vals]
    cat_counts = cat_counts/np.sum(cat_counts)

    for meth in outlier_detector.methods+[outlier_detector]:
        outlying_items, counts_num = np.unique(df_bag.loc[df_bag[f"is_{ meth.name}_outlier"], cat], return_counts=True)
        counts_num = counts_num/np.sum(counts_num)
        if cat == "actor":
            outlying_items = [int(act_name.split("_")[1]) for act_name in outlying_items]

        counts = {c:0 for c in np.unique(cat_vals)}
        for outlying_item, out_count, cc in zip(outlying_items, counts_num, cat_counts):
            # Normalize on category count
            # e.g.: More values of fear --> more outliers of fear
            counts[outlying_item] = out_count/cc

        plot_polar_closed(counts.keys(), counts.values(), axes[i], label=meth.name, color=code_map[meth.name])

axes[i].legend()
axes[0].set_title("emotion");
axes[1].set_title("actor");

## Save results

In [ ]:
df_bag = df_bag.reset_index(drop=True)
df_bag.to_csv(LABELLED_OUTLIERS, index=False)

# Reove outliers
df_outliers_removed = df_bag.loc[np.logical_not(df_bag.is_aggregatedOD_outlier)]
df_outliers_removed.to_csv(OUTLIERS_REMOVED, index=False)